In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N = int(input())
  M = int(input())
  G = [[] for _ in range(N+1)]
  for _ in range(M) :
    a, b, w = map(int, input().split())
    G[a].append((b, w))

  S, E = map(int, input().split())
  DP = [0] * (N+1) #시작점에서 해당 정점으로 오는데 걸리는 최대 시간
  DP2 = [[] for _ in range(N+1)]# DP[i]인 i번째 정점을 도착점으로 두는 정점

  Q = [(S, 0)]
  while Q :
    nextQ = []
    while Q :
      u, w1 = Q.pop()
      for v, w2 in G[u] :
        if DP[v] < w1 + w2 :
          DP[v] = w1 + w2
          DP2[v] = [u]
          nextQ.append((v, DP[v]))
        elif DP[v] == w1 + w2 :
          DP2[v].append(u)
          # 이미 discovery 된 정점이므로 경우이므로 그 다음 경로를 nextQ에 넣지 않는다.
    Q = nextQ
  
  Q = [E]
  vis = [False] * (N+1)
  vis[E] = True
  while Q : #DP2에 기록한 내용을 토대로 역방향으로 다시 탐색
    u = Q.pop()
    vis[u] = True
    for v in DP2[u] :
      if not vis[v] :
        Q.append(v)

  print(DP[E])
  print(sum(vis))

sol()

### 아이디어
- 지문 이해를 너무 늦게 했다. $S \to E$ 의 최장 경로의 가중치와 $S \to E$ 의 최장 경로에 속하는 모든 간선의 집합을 $DP$ 라고 할 때 $|DP|$ 를 구하는 문제이다.
- DP 2개를 이용하여 하나엔 최장 경로의 가중치 합을, 다른 하나엔 최장 경로의 개수를 저장하는 것을 목표로했다.
  - 3% WA
  - 그러한 정점의 개수를 세고 앉아있었다. 간선의 수를 세어야 한다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N = int(input())
  M = int(input())
  G = [[] for _ in range(N+1)]
  for _ in range(M) :
    a, b, w = map(int, input().split())
    G[a].append((b, w))

  S, E = map(int, input().split())
  DP = [0] * (N+1) #시작점에서 해당 정점으로 오는데 걸리는 최대 시간
  DP2 = [[] for _ in range(N+1)]# DP[i]인 i번째 정점을 도착점으로 두는 정점
  # DP2[S] = [-1]

  Q = [(S, 0)]
  while Q :
    nextQ = []
    while Q :
      u, w1 = Q.pop()
      for v, w2 in G[u] :
        if DP[v] < w1 + w2 :
          DP[v] = w1 + w2
          DP2[v] = [u]
          nextQ.append((v, DP[v]))
        elif DP[v] == w1 + w2 :
          DP2[v].append(u)
          # 이미 discovery 된 정점이므로 경우이므로 그 다음 경로를 nextQ에 넣지 않는다.
    Q = nextQ
  
  Q = [E]
  cnt = 0
  vis = [False] * (N+1)
  while Q : #DP2에 기록한 내용을 토대로 역방향으로 다시 탐색
    u = Q.pop()
    cnt += len(DP2[u])
    for v in DP2[u] :
      if not vis[v] :
        vis[v] = True
        Q.append(v)

  print(DP[E])
  print(cnt)

sol()

- 정답을 맞았지만, 정해가 이것이 아닌듯 하다. 위상정렬?
- 간선의 개수를 세는 구현을 이 코드에서 참고하면 좋을 것 같다.

### 위상정렬 풀이
- 위상정렬은 그래프가 DAG일때 사용할 수 있다. 그러한 그래프에서 위상정렬을 수행하면 dp 진행 순서를 구할 수 있다.
  - 진입 차수가 0일때에만 다음 정점을 추가하므로, 그래프를 훨씬 더 효율적으로 탐색할 수 있게 된다. 위의 코드는 만약 간선수가 더 많았다면 TLE가 났을 것이다.
- 그리고 가중치 배열 W를 따로 만들었었는데, N * N이 굉장히 커서 배열을 초기화 하는 시간이 오래 걸린게 있었다. 유의하는 것이 좋겠다.
  - https://www.acmicpc.net/source/67236591

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N = int(input())
  M = int(input())
  G = [[] for _ in range(N)]
  IN = [0] * N
  for _ in range(M) :
    a, b, w = map(int, input().split())
    G[a-1].append((b-1, w))
    IN[b-1] += 1
  
  S, E = map(lambda x: int(int(x)-1), input().split())
  DP = [0] * N #현재 노드까지의 최대 가중치
  DP2 = [[] for _ in range(N)] #최대 가중치를 만드는 경로
  Q = [S] 
  nextQ = []
  while Q :
    while Q:
      u = Q.pop()
      for v, w in G[u] :
        cost = DP[u] + w
        if DP[v] < cost :
          DP[v] = cost
          DP2[v] = [u]
        elif DP[v] == cost :
          DP2[v].append(u)
        IN[v] -= 1
        if IN[v] == 0 :
          nextQ.append(v)
    Q = nextQ
    nextQ = []

  cnt = 0
  Q = [E] 
  vis = [False] * N
  vis[E] = True
  while Q :
    u = Q.pop()
    cnt += len(DP2[u])
    for v in DP2[u] :
      if not vis[v] :
        vis[v] = True
        Q.append(v)
  print(DP[E])
  print(cnt)

sol()